In [ ]:
"""
WEB SCRAPPING

#To install requests-html and jupyter in virtualenv
pipenv install requests -html 

#If you are creating a new folder. Install Jupyternotebook
pipenv install jupyter notebook

#To start jupyter notebook from virtualenv
pipenv run jupyter notebook

#For additional info go to: 
http://python-requests.org/

#print(response.content) - will get you the content of the website

#alway add "." before the class for e.g. ".subtext"

#Reference: https://github.com/pycampers/foundation_bootcamp/blob/master/6th_sept_web_scraping.ipynb

#first = True - means just give the frist value not the list
"""

In [ ]:
"""
Steps to download the subtitle of a movie:

1. Ask the user movie name
2. Once we have the movie name, we go to the website and search for the movie and get the response.
3. Get all the results
4. Exit if no result is found
5. take movie search results convert them into a dictionary
6. ask user to select the movie from the list
7. get the selected movie page
8. find a list of subtitles for that movie
9. convert html subtitles to dictionary
10. select subtitle based on language and rating
11. get the zip file url from the movie page
12. download the zip file

"""

In [2]:
from urllib.parse import quote
from requests_html import HTMLSession
import requests


def url_to_response(url):
    session = HTMLSession()
    response = session.get(url)
    return response


def search_for_subtitle(query):
    website_url = "https://yts-subs.com/"
    query_encoded = quote(query)
    website_search_url = website_url + "search/" + query_encoded

    response = url_to_response(website_search_url)
    return response


def search_result_to_movie_info(single_search_result):
    poster = single_search_result.find('.media-object', first=True).attrs["src"]
    title = single_search_result.find(".media-heading", first=True).text

    year_raw = single_search_result.find(".movinfo-section")[0].text
    year = int(year_raw.split()[0])
    movie_page_raw = single_search_result.find(".media-body", first=True).find("a", first=True)
    movie_page = movie_page_raw.absolute_links.pop()

    movie_info = {"title":title,
                 "year": year,
                 "poster": poster,
                 "movie_page": movie_page}

    return movie_info


def select_movie_number(movie_info_list):
    for i in range(len(movie_info_list)):
        single_movie_info = movie_info_list[i]
        print(f'{i+1} : {single_movie_info["title"]} - {single_movie_info["year"]}')

    selected_movie = int(input("Enter the movie number: "))
    return movie_info_list[selected_movie - 1]
              

def subtitle_html_to_dict(single_subtitle):
    subtitle_language = single_subtitle.find(".sub-lang", first=True).text

    download_cell = single_subtitle.find('.download-cell', first=True)
    download_page_link = download_cell.find("a", first=True).absolute_links.pop()

    subtitle_rating = int(single_subtitle.find(".rating-cell", first=True).text)

    subtitle_dict = {"language": subtitle_language,
                    "download_page": download_page_link,
                    "rating": subtitle_rating}

    return subtitle_dict


def filter_subtitles(subtitle_info_list, filter_language="English"):
    results = []
    for single_subtitle in subtitle_info_list:
        if single_subtitle['language'] == filter_language:
            results.append(single_subtitle)

    if len(results) == 0:
        selected_subtitle = None
    else:
        selected_subtitle = results[0]

    return selected_subtitle


def download_page_to_zip_url(final_subtitle):
    response = url_to_response(final_subtitle["download_page"])
    download_link = response.html.find('.download-subtitle', first=True).attrs["href"]
    return download_link


def download_zip_file(zip_url):
    zip_file_data = requests.get(zip_url).content
    filename = zip_url.split("/")[-1]

    with open(filename, "wb") as zip_file:
        zip_file.write(zip_file_data)
    
    return 0


def main():
    query = input("Enter movie name: ") # take movie name from user
    response = search_for_subtitle(query) # search for movie on website
    search_results = response.html.find('.media-movie-clickable') # get all the results

    if len(search_results) > 0:
        print(f"found {len(search_results)} subtitles. ")
    else:
        print("No results found.")
        return 0 # exit is no result is found

    # take search results convert them into a dict
    movie_info_list = list(map(search_result_to_movie_info, search_results))

    # Ask user to select a movie from the list
    selected_movie = select_movie_number(movie_info_list)

    # get the selected movie page
    movie_page_response = url_to_response(selected_movie["movie_page"])

    # find the list of subtitles for that movie
    subtitle_list = movie_page_response.html.find('.high-rating')
 
    # convert html subtitles to dict
    subtitle_info_list = list(map(subtitle_html_to_dict, subtitle_list))
    
    # select subtitle based on language and rating
    final_subtitle = filter_subtitles(subtitle_info_list)
    
    # get the zip file url from the movie page
    zip_url = download_page_to_zip_url(final_subtitle)
    
    # download the zip file
    download_zip_file(zip_url)
    print("Downloaded Subtitle in the current Directory.")
          

main()

ModuleNotFoundError: No module named 'requests_html'

NameError: name 'SubtitleDownloader' is not defined